In [1]:
import pandas as pd
import numpy as np
import pprint

In [2]:
# base variables
csv_files = ["ALP", "BS", "DR", "EC", "HD", "JF", "JR", "SS"]
file_suffix = ".csv"

BASE_DIR = "~/Research/wheelchair/data/raw/Max"
OUTPUT_DIR = "~/Research/wheelchair/data/processed/"

# variables for glove types
HYB = "HYB"
PLA = "PLA"

In [23]:
# configuration setup

initials = csv_files[5]
material = PLA
input_file = f"{BASE_DIR}/{material}/{initials}25{material}.csv"
output_file = f"{OUTPUT_DIR}/{material}/{initials}25{material}_per_cycle.csv"

### Helper functions and config

In [24]:
def mean_positive(x):
    positives = x[x > 0]
    if len(x) == 0:
        return np.nan
    return positives.mean()

def mean_negative(x):
    negatives = x[x < 0]
    if len(x) == 0:
        return np.nan
    return negatives.mean()

In [25]:
### Build the parameter dictionary
# tangential_force_R[N], radial_force_R[N], axle_force_R[N], moment_z_R[Nm], 'power_z[W]'
xy_force_params = {
    "tangential_force":"N" , 
    "radial_force": "N", 
    "axle_force": "N"
}
z_params = {
    "moment_z_R": "Nm", 
    "power_z": "W"
}

sides = ["L", "R", "avg"]

# build up a dictionary containing the parameters we want to calculate
agg_dict = {}
for col, unit in xy_force_params.items():
    for side in sides:
        col_name = f"{col}_{side}" 
        og_column = f"{col_name}[{unit}]"
        agg_dict[f"{col_name}_pos[{unit}]"] = (og_column, mean_positive)
        agg_dict[f"{col_name}_neg[{unit}]"] = (og_column, mean_negative)
        agg_dict[f"{col_name}_pos_peak[{unit}]"] = (og_column, "max")
        agg_dict[f"{col_name}_neg_peak[{unit}]"] = (og_column, "min")

for col, unit in z_params.items():
    og_column = f"{col}[{unit}]"
    agg_dict[f"{col}_pos[{unit}]"] = (og_column, mean_positive)
    agg_dict[f"{col}_neg[{unit}]"] = (og_column, mean_negative)
    agg_dict[f"{col}_pos_peak[{unit}]"] = (og_column, "max")
    agg_dict[f"{col}_neg_peak[{unit}]"] = (og_column, "min")

### Clean the data
Also add variables

In [26]:
# clean the data
raw_df = pd.read_csv(input_file)

# only use named columns and when the hand cycle is touching
df = raw_df.drop(columns=[c for c in raw_df.columns if "Unnamed" in c])
df = df[df['theta_cop_R[deg]'].notna()]

# add the power[W] calculation for future calculations down the line
df['power_z[W]'] = df['gyro_z_R[rad/s]']*df['moment_z_R[Nm]']

# add the power[W] calculation for future calculations down the line
df['total_force_R[N]'] = np.sqrt(
    df["tangential_force_R[N]"]**2 +
    df["radial_force_R[N]"]**2 +
    df["axle_force_R[N]"]**2
)

df['total_force_L[N]'] =  np.sqrt(
    df["tangential_force_L[N]"]**2 +
    df["radial_force_L[N]"]**2 +
    df["axle_force_L[N]"]**2
)

# Set missing values to 0
# i.e. if they lift their hand up earlier, the force is not recorded, and is set to 0
df['total_force_R[N]'] = df['total_force_R[N]'].fillna(0)
df['total_force_L[N]'] = df['total_force_L[N]'].fillna(0)
df['total_force_average[N]'] = (df['total_force_R[N]'] + df['total_force_L[N]'])/2.0

# Add the average values across the R and L sides too
for param in xy_force_params:
    avg_param_name = f"{param}_avg[N]"
    l_param_name =f"{param}_L[N]"
    r_param_name=f"{param}_R[N]"
    df[avg_param_name]=(df[l_param_name] + df[r_param_name]) / 2.0


In [27]:
# 1. Total force (N)
total_force = df.groupby("cycle[count]")[
    ['total_force_R[N]', 
     'total_force_L[N]',
     'total_force_average[N]']].mean()

In [28]:
# 2-5: split into R&L along with positive, negative values positive and negative values
"""
2. Tangential Force
3. Radial Force
4. Axle Force 
5. Torque
6. Power
"""
avg_kinetics = df.groupby("cycle[count]").agg(**agg_dict)

In [29]:
# 2-5: get the average across the L and R sides

xy_force_params = {
    "tangential_force":"N" , 
    "radial_force": "N", 
    "axle_force": "N"
}

In [30]:
kinetics_df = pd.concat([total_force, avg_kinetics], axis=1)
kinetics_df.describe()

,total_force_R[N],total_force_L[N],total_force_average[N],tangential_force_L_pos[N],tangential_force_L_neg[N],tangential_force_L_pos_peak[N],tangential_force_L_neg_peak[N],tangential_force_R_pos[N],tangential_force_R_neg[N],tangential_force_R_pos_peak[N],...,axle_force_avg_pos_peak[N],axle_force_avg_neg_peak[N],moment_z_R_pos[Nm],moment_z_R_neg[Nm],moment_z_R_pos_peak[Nm],moment_z_R_neg_peak[Nm],power_z_pos[W],power_z_neg[W],power_z_pos_peak[W],power_z_neg_peak[W]
count,25.000000,25.000000,25.000000,25.000000,23.000000,25.000000,25.000000,25.000000,22.000000,25.000000,...,25.000000,25.000000,25.000000,18.000000,25.000000,25.000000,18.000000,25.000000,25.000000,25.000000
mean,144.817633,126.764206,135.790920,93.842259,-8.601978,142.457179,-12.844271,88.086281,-12.411645,142.994620,...,135.419852,6.532648,12.655771,-2.096257,19.454521,-3.242286,49.195503,-239.925124,82.260418,-379.573576
std,21.613533,22.750552,21.206644,20.190434,6.777706,21.289607,13.055641,22.001091,9.155439,22.965666,...,22.744937,2.672632,3.972236,1.370754,4.789846,4.754685,33.971250,44.105867,115.555814,61.391083
min,83.387486,106.662033,95.809056,63.452047,-31.601685,110.359788,-45.857147,30.880628,-30.465437,70.273497,...,105.283749,1.465996,4.972363,-5.193572,11.712387,-18.902616,1.423444,-277.702005,-42.435269,-439.295378
25%,133.531770,110.631356,122.081563,80.182699,-11.188020,128.821658,-24.199347,74.180092,-18.997921,131.290910,...,120.895744,5.330963,10.155002,-2.691072,15.892870,-5.481859,27.920142,-260.610977,-0.478399,-413.782205
50%,141.111466,119.071641,134.076841,86.811835,-7.251617,141.912925,-11.939863,82.074301,-8.662998,140.932801,...,133.559092,6.983466,11.237839,-1.934478,17.591191,-2.205808,39.565256,-251.840077,51.028924,-396.443353
75%,151.554069,137.006937,144.280503,102.387588,-5.275490,157.752489,-4.033655,99.911273,-5.941112,159.910917,...,146.453638,8.142709,14.620482,-1.200884,21.540541,0.070345,67.490167,-241.005520,124.249852,-385.311519
max,201.778625,199.745014,200.761819,140.477159,-0.452163,185.391904,15.066934,140.945568,-0.979519,183.644102,...,201.735953,10.785422,23.123057,-0.110542,31.921404,2.617826,133.939201,-87.270481,486.254558,-158.585020


In [31]:
kinetics_df

,total_force_R[N],total_force_L[N],total_force_average[N],tangential_force_L_pos[N],tangential_force_L_neg[N],tangential_force_L_pos_peak[N],tangential_force_L_neg_peak[N],tangential_force_R_pos[N],tangential_force_R_neg[N],tangential_force_R_pos_peak[N],...,axle_force_avg_pos_peak[N],axle_force_avg_neg_peak[N],moment_z_R_pos[Nm],moment_z_R_neg[Nm],moment_z_R_pos_peak[Nm],moment_z_R_neg_peak[Nm],power_z_pos[W],power_z_neg[W],power_z_pos_peak[W],power_z_neg_peak[W]
cycle[count],,,,,,,,,,,,,,,,,,,,,
1,201.778625,199.745014,200.761819,140.477159,NaN,185.391904,15.066934,140.945568,NaN,183.644102,...,181.649120,10.785422,23.123057,NaN,31.921404,2.024921,NaN,-87.270481,-0.478399,-158.585020
2,181.244020,168.079727,174.661874,139.685072,-0.888783,182.914197,-0.888783,123.192639,NaN,171.140816,...,154.307215,1.741078,20.156045,NaN,28.514713,1.592804,NaN,-182.542813,-11.584213,-264.396590
3,169.320203,165.361745,167.340974,127.635418,-5.592887,176.618144,-5.592887,117.955495,-2.115315,165.598489,...,138.324531,4.578957,18.972200,-0.467290,26.248609,-0.467290,4.934174,-224.817826,4.934174,-316.827922
4,157.068684,138.779998,147.924341,115.380898,-3.054711,160.866577,-4.033655,106.892497,-0.979519,156.615561,...,134.395548,2.134477,16.958799,-0.110542,24.017679,-0.165812,1.423444,-236.404031,2.135872,-341.378853
5,162.638048,139.833219,151.235633,108.159307,-0.799593,159.770971,-1.065108,109.749943,-11.144403,163.251901,...,118.543988,1.465996,16.084762,NaN,24.113146,0.638127,NaN,-251.024966,-9.377062,-378.764629
6,148.601748,140.564380,144.583064,102.340734,NaN,157.752489,0.494283,99.899031,NaN,171.492361,...,120.967386,6.983466,14.620482,NaN,23.982506,2.617826,NaN,-247.643023,-42.435269,-411.678428
7,151.554069,137.006937,144.280503,105.884470,-7.251617,159.240489,-7.251617,101.412174,-15.957222,160.501027,...,108.694964,6.965014,14.647392,NaN,21.540541,0.879308,NaN,-265.595671,-15.312984,-390.574968
8,156.466807,112.191326,134.329067,99.327487,-0.452163,151.314361,-0.452163,99.911273,-5.973729,151.619437,...,105.283749,8.464901,13.342808,-1.547583,21.279261,-3.034870,28.555889,-256.543512,55.997449,-413.782205
9,151.092243,126.651351,138.871797,102.387588,-7.295123,148.565587,-7.295123,95.179710,-5.860703,141.293106,...,120.884482,6.104461,13.792579,-2.321374,20.706455,-5.592401,44.949989,-277.316559,108.150916,-418.465720


In [32]:
kinetics_df.to_csv(output_file)